In [ ]:
#This code is for generating x_train and y_train patches from prospective high b-value DWI for training DnCNN 

import nibabel as nib
import numpy as np
import os
import os.path
import cv2
import matplotlib.pyplot as plt

patch_size, stride = 40, 10
aug_times =10
#num_threads = 16  
imx =[]
imy =[]
x_patches = []
y_patches = []
scales = [1, 0.9, 0.8, 0.7]

def data_aug(img_data, mode=0):
        if mode == 0:
           return img_data
        elif mode == 1:
           return np.flipud(img_data)
        elif mode == 2:
           return np.rot90(img_data,axes=(0, 1))
        elif mode == 3:
           return np.flipud(np.rot90(img_data,axes=(0, 1)))
        elif mode == 4:
           return np.rot90(img_data, k=2,axes=(0, 1))
        elif mode == 5:
           return np.flipud(np.rot90(img_data, k=2,axes=(0, 1)))
        elif mode == 6:
           return np.rot90(img_data, k=3,axes=(0, 1))
        elif mode == 7:
           return np.flipud(np.rot90(img_data, k=3,axes=(0, 1)))
    

                    
        

In [ ]:
# Read X_train

for dirpath, dirnames, filenames in os.walk("Path for high b-value DWI data"):
    #for filename in [f for f in filenames if f.endswith("b3000_avg1_full_128.nii" )]:
    #for filename in [f for f in filenames if f.endswith("b3000_avg2_full_128.nii" )]:
    for filename in [f for f in filenames if f.endswith("b3000_avg4_full_128.nii" )]:
        img_path=os.path.join(dirpath, filename)
        img = nib.load(os.path.join(dirpath, filename))
        img_data2 = np.array(img.get_data())          
        print(img_data2.shape)
        img_data2=img_data2.astype('float32')   

        slice_total=img_data2.shape[2]
        for num_slice in range(slice_total):
            img_data2[:,:,num_slice]=img_data2[:,:,num_slice]
            imgx_data=img_data2[:,:,num_slice]
            imx.append(imgx_data)
  

In [ ]:
 # Read Y_train

 for dirpath, dirnames, filenames in os.walk("."):
     for filename in [f for f in filenames if f.endswith("b3000_avg10_full_128.nii" )]:
         img_path=os.path.join(dirpath, filename)
         img = nib.load(os.path.join(dirpath, filename))
         img_data2 = np.array(img.get_data())   
         img_data2=img_data2.astype('float32')       
         print(img_data2.shape)             
         slice_total=img_data2.shape[2]
         for num_slice in range(slice_total):
             #global_thresh = threshold_otsu(img_data2[:,:,num_slice])
            # binary_global = img_data2[:,:,num_slice] > global_thresh
             img_data2[:,:,num_slice]=img_data2[:,:,num_slice]
             imgy_data=img_data2[:,:,num_slice]
          #if np.sum(imgy_data>45):
             imy.append(imgy_data)


In [ ]:
# Scaling , data augmentantion and generate patches

h, w = imgx_data.shape
for num_slice in range(slice_total):
    for s in scales:
        h_scaled, w_scaled = int(h*s),int(w*s)
        imgx_scaled = cv2.resize(imx[num_slice], (h_scaled,w_scaled), interpolation=cv2.INTER_CUBIC)
        imgy_scaled = cv2.resize(imy[num_slice], (h_scaled,w_scaled), interpolation=cv2.INTER_CUBIC)

    # extract patches
        for i in range(0, h_scaled-patch_size+1, stride):
            for j in range(0, w_scaled-patch_size+1, stride):
                  x_train = imgx_scaled[i:i+patch_size, j:j+patch_size]
                  y_train = imgy_scaled[i:i+patch_size, j:j+patch_size]
                #print(x.shape)
    # data aug
                  for k in range(0, aug_times):
                      mode=np.random.randint(0,8)                              
                      x_aug_train = data_aug(x_train,mode)
                      y_aug_train = data_aug(y_train,mode)
                      #if np.sum(x_aug_train>0):
                      x_patches.append(x_aug_train)
                      #if np.sum(y_aug_train>0):
                      y_patches.append(y_aug_train)
    # save to .npy
resx = np.asarray(x_patches)
resy = np.asarray(y_patches)

print(len(resx))
print(len(resy))


197100
197100


In [ ]:
# Save x_train patches
save_dir = '.'
resx = np.asarray(x_patches)
print('Shape of result = ' + str(resx.shape))
print('Saving data...')
np.save('./x_train_avg4_128_23.npy', resx)
print('Done.')

In [ ]:
# Save y_train patches
save_dir = '.'
resy = np.asarray(y_patches)
print('Shape of result = ' + str(resy.shape))
print('Saving data...')
np.save('./y_train_avg10_128_foravg4_23.npy', resy)
print('Done.')